In [ ]:
from __future__ import division
%matplotlib inline
import matplotlib.pyplot as plt
from postBornCrossSpectra import PostBorn_Bispec
import numpy as np
import Cosmology as C
import pickle
from scipy.integrate import simps
from scipy.interpolate import RectBivariateSpline
from classy import Class

import copy
import kernels
plt.style.use(['seaborn-colorblind','paper'])
plt.style.use('classic')

In [ ]:
plotpath = './CrossPostBorn/results/plots/vanessa/'

In [ ]:
zmin = 1e-5
zmax = 1090.

kmin = 1e-4
kmax = 50

cosmo = C.Planck2015

a     = np.linspace(1./(1.+zmin),1./(1.+zmax),500)  
z     = 1./a-1.

data  = C.CosmoData(cosmo[1],z)

LSST_bin  = 'all'

L     = np.logspace(1,4,100)


In [ ]:
#compute 31 term seperately
def get_31_term(L,L1,M):
    res=[]
    for L_ in L:
        res+=[simps(M(L_,L1,grid=False)/L1,L1)]

    res= np.asarray(res)
    res*=-1/(2*np.pi)*L**2
    return res

In [ ]:
#compute correction following Krause&Hirata
def get_regularized_sum_of_terms(L,L1,phi,M):
    nu = np.cos(phi)
    res=[]
    for LL in L:
        resL1=[]
        for LL1 in L1:
            L_= LL*np.sqrt(1.+(LL1/LL)**2-2.*LL1/LL*nu)
            integrand = LL**2*LL1**3*(LL*nu-LL1)**2*(LL-LL1*nu)**2*(M(LL,LL1,grid=False)/LL**4/LL1**4-M(L_,LL1,grid=False)/L_**4/LL1**4)
            resL1+=[simps(integrand,phi)]
        res+=[simps(resL1,L1)]
    res=4.*np.asarray(res)/(2*np.pi)**2
    return res

def get_second_sum_of_terms(L,L1,M):
    res=[]
    for LL in L:
        integrand = L1*(27./4.*LL*L1+LL**2*L1**6)*(M(LL,L1,grid=False)/LL**4/L1**4)
        res+=[simps(integrand,L1)]   
    return 4.*np.asarray(res)/4./np.pi

In [ ]:
#compute correction following Krause&Hirata
def get_regularized_sum_of_terms(L,L1,phi,M):
    nu = np.cos(phi)
    res=[]
    for LL in L:
        resL1=[]
        for LL1 in L1:
            L_= LL*np.sqrt(1.+(LL1/LL)**2+2.*LL1/LL*nu)
            integrand =LL**2*LL1**3*(LL1*nu+LL)**2*(LL1+LL*nu)**2*(M(L_,LL1,grid=False)/L_**4/LL1**4-M(LL,LL1,grid=False)/LL**4/LL1**4)
            resL1+=[simps(integrand,phi)]
        res+=[simps(resL1,L1)]
    res=4.*np.asarray(res)/(2*np.pi)**2
    return res

def get_second_sum_of_terms(L,L1,M):
    nu = np.cos(phi)
    res=[]
    for LL in L:
        resL1=[]
        for LL1 in L1:
            integrand = LL1*(LL**2*LL1**2*(LL1*nu+LL)**2*(LL1+LL*nu)**2-LL**4*(LL*LL1*nu)**2)*(M(LL,LL1,grid=False)/LL**4/LL1**4)
            resL1+=[simps(integrand,phi)]
        res+=[simps(resL1,L1)]
    res=4.*np.asarray(res)/(2*np.pi)**2
    return res

In [ ]:
#test functions for angular integration
def test_ang_integration(L,L1,phi):
    nu = np.cos(phi)
    res1=[]
    for LL in L:
        resL1=[]
        for LL1 in L1:
            integrand = (LL*nu-LL1)**2*(LL-LL1*nu)**2/LL**4/LL1**4
            resL1+=[simps(integrand,phi)]
        res1+=[simps(resL1,L1)]
    
    res2=[]
    for LL in L:
        # first term dominates, second term gives sub-percent corrections
        integrand = (np.pi*(LL**4+L1**4+4*LL**2*L1**2)+(2*np.pi+3./4.*np.pi)*(LL**2*L1**2))/LL**4/L1**4
        res2+=[simps(integrand,L1)]
    return np.asarray(res1),np.asarray(res2)

In [ ]:
phi=np.linspace(0,2.*np.pi,800)
L1=np.logspace(-1,5,400)

#sub-percent difference in angular integrations, so looks like L-scaling of M matrix is screwed up?
res1, res2 = test_ang_integration(L,L1,phi)
plt.semilogx(L,(res1-res2)/res1*100)
plt.plot(L,(res1-res2)/res2*100)

In [ ]:
# first test for CMB lensing auto spectrum, change second kernel to initialize for cross
first_kernel  = kernels.CMB_lens(data.chi_cmb,data)
simple_kernel = kernels.CMB_lens(None,data)
second_kernel = kernels.CMB_lens(data.chi_cmb,data)#kernels.gal_clus(kernels.dNdz_LSST,kernels.simple_bias,data,LSST_bin)

PB1 = PostBorn_Bispec(data, zmin, data.z_cmb, first_kernel, second_kernel, simple_kernel, k_min=kmin,k_max=100, lmax=30000, acc=2)

In [ ]:
phi=np.linspace(0,2.*np.pi,800)
L1=np.logspace(-1,5,400)
plt.figure()
plt.loglog(PB1.ls,PB1.CL_born,label='(11) term')
reskk1 = get_regularized_sum_of_terms(L,L1,phi,PB1.Mstarsp)
reskk2 = get_second_sum_of_terms(L,L1,PB1.Mstarsp)
plt.loglog(L, -reskk1,label='integral 1')
plt.loglog(L, reskk2,ls='--',label='integral 2')
plt.loglog(L, reskk1+reskk2,ls='--',label='difference')
plt.loglog(L, -(reskk1+reskk2),ls='--',label='difference')
# #(31)-term, multiply by two to get (31)+(13)
# L1=np.logspace(-3,5,400)
# reskk = get_31_term(L,L1,PB1.Mstarsp)
# plt.loglog(L,-reskk,label='(31) term 2',ls='--')
# L1=np.logspace(-3,5,400)
# reskk = get_31_term(L1,L,PB1.Mstarsp)
# plt.loglog(L1,-reskk,label='(31) term 3',ls='--')
# L1=np.logspace(-6,5,400)
# reskk = get_22_term2(L1,L1,phi,PB1.Mstarsp)
# plt.loglog(L1,reskk,label='(22) term 2',ls='--')
plt.xlim(10,10000)
plt.ylim(1e-13,1e-6)
plt.legend(loc='best')
plt.ylabel('$C_L^{\kappa\kappa}$',fontsize=20)
plt.xlabel('$L$',fontsize=20)
plt.savefig(plotpath+'cl_kk_pB.pdf', bbox_inches='tight')
plt.show()

In [ ]:
# first test for CMB lensing auto spectrum, change second kernel to initialize for cross
first_kernel  = kernels.CMB_lens(data.chi_cmb,data)
simple_kernel = kernels.CMB_lens(None,data)
second_kernel = kernels.gal_clus(kernels.dNdz_LSST,kernels.simple_bias,data,LSST_bin)

PB2 = PostBorn_Bispec(data, zmin, data.z_cmb, first_kernel, second_kernel, simple_kernel, k_min=1e-4,k_max=200, lmax=60000, acc=1)

#(31)-term, multiply by two to get (31)+(13)


In [ ]:
plt.figure()
for L_ in [100,1000,10000]:
    dlnMdlnL1=L1*PB2.Mstarsp(L_,L1,dx=0,dy=1)/PB2.Mstarsp(L_,L1,dx=0,dy=0)
    plt.semilogx(L1,dlnMdlnL1[0],label='L=%d'%L_)
    
plt.legend()
plt.show()


In [ ]:
plt.figure()
for cc in [data.chi_cmb/10.,data.chi_cmb]:
    dlnM1dlnL1=L1*PB2.inner_int(cc,L1,dx=0,dy=1)/PB2.inner_int(cc,L1,dx=0,dy=0)
    plt.semilogx(L1,dlnM1dlnL1[0],label='chi=%d'%cc)
    
plt.legend(loc='best')
plt.show()

plt.figure()
for cc in [data.chi_cmb/10.,data.chi_cmb]:
    dlnM1dlnL1=PB2.inner_int(cc,L1,dx=0,dy=0)
    plt.semilogx(L1,dlnM1dlnL1[0],label='chi=%d'%cc)
    
plt.legend(loc='best')
plt.show()

In [ ]:

plt.figure()
plt.loglog(PB2.ls,PB2.CL_born,label='(11) term')
L1=np.logspace(-5,np.log10(60000),1000)
reskg = get_31_term(L,L1,PB2.Mstarsp)
plt.loglog(L,-reskg,label='(31) term')
L1=np.logspace(-1,np.log10(60000),1000)
reskg = get_31_term(L,L1,PB2.Mstarsp)
plt.loglog(L,-reskg,label='(31) term 2',ls='--')
plt.xlim(10,50000)
plt.legend()
plt.ylabel('$C_L^{\kappa g}$',fontsize=20)
plt.xlabel('$L$',fontsize=20)
plt.ylim(1e-12,1e-6)
plt.savefig(plotpath+'cl_kg_pB_L1min%d.pdf'%(min(L1)), bbox_inches='tight')

plt.show()

In [ ]:
PB2.Mstarsp(L,L1)

In [ ]:
# first test for CMB lensing auto spectrum, change second kernel to initialize for cross
first_kernel  = kernels.gal_lens((0,1),data, p_chi=kernels.p_delta(data,1.))
simple_kernel = kernels.CMB_lens(None,data)
second_kernel = kernels.gal_lens((0,1),data, p_chi=kernels.p_delta(data,1.))

PB3 = PostBorn_Bispec(data, zmin, data.z_cmb, first_kernel, second_kernel, simple_kernel, k_min=kmin,k_max=kmax, lmax=None, acc=1)

#(31)-term, multiply by two to get (31)+(13)
resgamgam = get_31_term(L,L1,PB3.Mstarsp)

In [ ]:

plt.figure()
plt.loglog(PB3.ls,PB3.ls**2*PB3.CL_born/(2*np.pi),label='(11) term')
L1=np.logspace(-5,4)
resgamgam = get_31_term(L,L1,PB3.Mstarsp)
plt.loglog(L,-L**2*resgamgam/(2*np.pi),label='(31) term')
L1=np.logspace(-1,4)
resgamgam = get_31_term(L,L1,PB3.Mstarsp)
plt.loglog(L,-L**2*resgamgam/(2*np.pi),label='(31) term 2', ls='--')
plt.xlim(10,10000)
plt.legend(loc='upper left')
plt.ylabel('$C_L^{\gamma \gamma}$',fontsize=20)
plt.xlabel('$L$',fontsize=20)
plt.grid()
plt.ylim(1e-10,1e-2)
plt.savefig(plotpath+'cl_gamgam_pB_L1min%d.pdf'%(min(L1)), bbox_inches='tight')
plt.show()

In [ ]:
L1 = np.logspace(-3,np.log10(20000),400)
phi = np.linspace(0,2.*np.pi,400)
L = L1
term2 = get_22_term(L,L1,phi,PB2.Mstarsp)
resgamgam = get_31_term(L,L1,PB3.Mstarsp)
plt.figure()
plt.loglog(PB3.ls,PB3.ls**2*PB3.CL_born/(2*np.pi),label='(11) term')
plt.loglog(L,-L**2*resgamgam/(2*np.pi),label='(31) term')
plt.loglog(L,L**2*term2/(2*np.pi),label='(22) term')
plt.xlim(10,10000)
plt.legend(loc='upper left')
plt.ylabel('$C_L^{\gamma \gamma}$',fontsize=20)
plt.xlabel('$L$',fontsize=20)
plt.grid()
plt.ylim(1e-10,1e-2)
plt.savefig(plotpath+'cl_gamgam_pB_L1min%d.pdf'%(min(L1)), bbox_inches='tight')
plt.show()